# Instituto Tecnológico y de Estudios Superiores de Monterrey
## Jesus Ramirez Delgado
## Modelación de Sistemas Multiagentes con Gráficas Computacionales
## Actividad Integradora 

## Parte 1: Sistemas Multiagentes
### Descripción del problema

¡Felicidades! Eres el orgulloso propietario de 5 robots nuevos y un almacén lleno de cajas. El dueño
anterior del almacén lo dejó en completo desorden, por lo que depende de tus robots organizar las
cajas en algo parecido al orden y convertirlo en un negocio exitoso.
Cada robot está equipado con ruedas omnidireccionales y, por lo tanto, puede conducir en las
cuatro direcciones. Pueden recoger cajas en celdas de cuadrícula adyacentes con sus
manipuladores, luego llevarlas a otra ubicación e incluso construir pilas de hasta cinco cajas. Todos
los robots están equipados con la tecnología de sensores más nueva que les permite recibir datos
de sensores de las cuatro celdas adyacentes. Por tanto, es fácil distinguir si un campo está libre, es
una pared, contiene una pila de cajas (y cuantas cajas hay en la pila) o está ocupado por otro robot.
Los robots también tienen sensores de presión equipados que les indican si llevan una caja en ese
momento.
Lamentablemente, tu presupuesto resultó insuficiente para adquirir un software de gestión de
agentes múltiples de última generación. Pero eso no debería ser un gran problema ... ¿verdad? Tu
tarea es enseñar a sus robots cómo ordenar su almacén. La organización de los agentes depende de
ti, siempre que todas las cajas terminen en pilas ordenadas de cinco.

* Realiza la siguiente simulación:
    * Inicializa las posiciones iniciales de las K cajas. Todas las cajas están a nivel de piso,
    es decir, no hay pilas de cajas.
    * Todos los agentes empiezan en posición aleatorias vacías.
    * Se ejecuta el tiempo máximo establecido.

* Deberás recopilar la siguiente información durante la ejecución:
    * Tiempo necesario hasta que todas las cajas están en pilas de máximo 5 cajas
    * Número de movimientos realizados por todos los robots.
    * Analiza si existe una estrategia que podría disminuir el tiempo dedicado, así como
    la cantidad de movimientos realizados. ¿Cómo sería? Descríbela.

In [147]:
# Imports que se utilizaran
import matplotlib
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from matplotlib.animation import FuncAnimation

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd
import time
import datetime
import random

%matplotlib inline
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

In [148]:
class Box(Agent):
    """ Define el agente caja con atributo de altura de pila. """
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.stack_height = 1
        self.is_carried = False

In [149]:
class RobotAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.carrying_box = None  # Comienza sin llevar ninguna caja

    def step(self):
        self.model.total_movements += 1  # Incrementa el contador de movimientos
        if not self.carrying_box:
            self.pick_up_box()
        else:
            self.move_to_stack()

    def pick_up_box(self):
        # Lógica para identificar y recoger una caja cercana
        neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False)
        boxes = [agent for agent in neighbors if isinstance(agent, Box) and agent.stack_height < 5 and not agent.is_carried]
        if boxes:
            chosen_box = self.random.choice(boxes)
            self.carrying_box = chosen_box
            self.model.grid.remove_agent(chosen_box)
            chosen_box.is_carried = True

    def move_to_stack(self):
        # Lógica para moverse hacia un lugar adecuado y depositar la caja
        potential_stacks = [(x, y) for x in range(self.model.grid.width) for y in range(self.model.grid.height)
                            if any(isinstance(agent, Box) and agent.stack_height < 5 for agent in self.model.grid.get_cell_list_contents((x, y)))]
        if potential_stacks:
            target_pos = self.random.choice(potential_stacks)
        else:
            target_pos = self.find_empty_space()

        self.model.grid.move_agent(self, target_pos)
        if self.carrying_box:
            self.model.grid.place_agent(self.carrying_box, target_pos)
            for agent in self.model.grid.get_cell_list_contents([self.pos]):
                if isinstance(agent, Box):
                    agent.stack_height += 1
                    break
            self.carrying_box.is_carried = False
            self.carrying_box = None

    def find_empty_space(self):
        # Encuentra una celda vacía en el grid para comenzar una nueva pila
        neighbors = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        empty_cells = [cell for cell in neighbors if self.model.grid.is_cell_empty(cell)]
        if empty_cells:
            return self.random.choice(empty_cells)
        return self.pos  # Si no hay celdas vacías, permanece en la posición actual

In [150]:
class RobotModel(Model):
    def __init__(self, width, height, n_boxes, n_robots):
        super().__init__()
        self.grid = MultiGrid(width, height, False)
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(model_reporters={"Grid": self.get_grid_state})
        self.current_id = 0
        self.total_movements = 0
        self.all_stacked = False
        self.steps_until_completion = 0

        for i in range(n_boxes):
            box = Box(self.next_id(), self)
            self.grid.place_agent(box, (np.random.randint(0, width), np.random.randint(0, height)))

        for i in range(n_robots):
            robot = RobotAgent(self.next_id(), self)
            self.grid.place_agent(robot, (np.random.randint(0, width), np.random.randint(0, height)))
            self.schedule.add(robot)

    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)
        if not self.all_stacked:
            self.steps_until_completion += 1
            if all(box.stack_height == 5 for cell in self.grid.coord_iter() for box in cell[0] if isinstance(box, Box)):
                self.all_stacked = True  # Comprobar si todas las cajas están apiladas a 5

    def get_grid_state(self):
        grid = np.zeros((self.grid.width, self.grid.height))
        for cell_content, (x, y) in self.grid.coord_iter():
            if cell_content:
                for content in cell_content:
                    if isinstance(content, RobotAgent) and content.carrying_box:
                        grid[x][y] = 5
                    elif isinstance(content, Box):
                        grid[x][y] = content.stack_height
        return grid

    def next_id(self):
        """ Retorna el siguiente ID único para los agentes, incrementando current_id """
        self.current_id += 1
        return self.current_id

In [151]:
# Configuración y ejecución del modelo
WIDTH = 3
HEIGHT = 3
N_BOXES = 2
N_ROBOTS = 3
model = RobotModel(WIDTH, HEIGHT, N_BOXES, N_ROBOTS)
START_TIME = time.time()
TIME_LIMIT = 15 # tiempo en segundos

steps = 0
while not model.all_stacked and (time.time() - START_TIME) < TIME_LIMIT:
    model.step()
    steps += 1

# Resultados
print(f'Tiempo hasta que todas las cajas están apiladas o hasta el límite de tiempo: {min(model.steps_until_completion, TIME_LIMIT)} segundos.')
print(f'Número total de movimientos realizados por todos los robots: {model.total_movements}')

Tiempo hasta que todas las cajas están apiladas o hasta el límite de tiempo: 15 segundos.
Número total de movimientos realizados por todos los robots: 6856257


In [152]:
# Crear una figura y un eje para la animación
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xticks([])
ax.set_yticks([])
ax.set_title('Simulación de Robots Organizando Cajas')

# Iniciar la animación con el estado inicial del grid
data = model.datacollector.get_model_vars_dataframe()
initial_grid = data.iloc[0, 0]
im = ax.imshow(initial_grid, cmap='viridis', vmin=0, vmax=5, interpolation='none')

def update(frame_number):
    grid = data.iloc[frame_number, 0]
    im.set_array(grid)
    return im,

# Crear la animación
ani = FuncAnimation(fig, update, frames=steps, blit=True, repeat=False)

# Mostrar la animación
plt.close(fig)  # Cerrar la figura para evitar la salida estática en Jupyter
ani

KeyboardInterrupt: 